In [1]:
from pandas.io.json import json_normalize # transform json into pandas dataframe
import matplotlib.cm as cm #plotting
import matplotlib.colors as colors
import matplotlib.pyplot as py
import json
import requests
import numpy as np  # data vectors
import pandas as pd # data analysis
from police_api import PoliceAPI
api = PoliceAPI()

In [2]:
forces = api.get_forces()
forces

[<Force> Avon and Somerset Constabulary,
 <Force> Bedfordshire Police,
 <Force> Cambridgeshire Constabulary,
 <Force> Cheshire Constabulary,
 <Force> City of London Police,
 <Force> Cleveland Police,
 <Force> Cumbria Constabulary,
 <Force> Derbyshire Constabulary,
 <Force> Devon & Cornwall Police,
 <Force> Dorset Police,
 <Force> Durham Constabulary,
 <Force> Dyfed-Powys Police,
 <Force> Essex Police,
 <Force> Gloucestershire Constabulary,
 <Force> Greater Manchester Police,
 <Force> Gwent Police,
 <Force> Hampshire Constabulary,
 <Force> Hertfordshire Constabulary,
 <Force> Humberside Police,
 <Force> Kent Police,
 <Force> Lancashire Constabulary,
 <Force> Leicestershire Police,
 <Force> Lincolnshire Police,
 <Force> Merseyside Police,
 <Force> Metropolitan Police Service,
 <Force> Norfolk Constabulary,
 <Force> North Wales Police,
 <Force> North Yorkshire Police,
 <Force> Northamptonshire Police,
 <Force> Northumbria Police,
 <Force> Nottinghamshire Police,
 <Force> Police Service of

In [3]:
def getIdName(records, name=None):
    ''' Get id force by name. If no name, return a dict of name:id mappings. '''
    if name:
        return {r.name:r.id for r in records if name.lower() in r.name.lower()}
    return {r.name:r.id for r in records}

In [4]:
getIdName(forces)

{'Avon and Somerset Constabulary': 'avon-and-somerset',
 'Bedfordshire Police': 'bedfordshire',
 'Cambridgeshire Constabulary': 'cambridgeshire',
 'Cheshire Constabulary': 'cheshire',
 'City of London Police': 'city-of-london',
 'Cleveland Police': 'cleveland',
 'Cumbria Constabulary': 'cumbria',
 'Derbyshire Constabulary': 'derbyshire',
 'Devon & Cornwall Police': 'devon-and-cornwall',
 'Dorset Police': 'dorset',
 'Durham Constabulary': 'durham',
 'Dyfed-Powys Police': 'dyfed-powys',
 'Essex Police': 'essex',
 'Gloucestershire Constabulary': 'gloucestershire',
 'Greater Manchester Police': 'greater-manchester',
 'Gwent Police': 'gwent',
 'Hampshire Constabulary': 'hampshire',
 'Hertfordshire Constabulary': 'hertfordshire',
 'Humberside Police': 'humberside',
 'Kent Police': 'kent',
 'Lancashire Constabulary': 'lancashire',
 'Leicestershire Police': 'leicestershire',
 'Lincolnshire Police': 'lincolnshire',
 'Merseyside Police': 'merseyside',
 'Metropolitan Police Service': 'metropolita

In [7]:
getIdName(forces, 'Thames Valley Police')

{'Thames Valley Police': 'thames-valley'}

In [8]:
force = api.get_force('thames-valley')

In [9]:
def getNameId(records, rid=None):
    ''' Get record name by id. '''
    if rid:
        return {r.name:r.id for r in records if rid == r.id}
    return {r.name:r.id for r in records}

In [10]:
areas = getNameId(force.neighbourhoods)
areas

{' Wing and Ivinghoe': 'N416',
 'Abbey / Battle': 'N464',
 'Abingdon Outer': 'N375',
 'Abingdon Town': 'N376',
 'Amersham': 'N430',
 'Ascot': 'N390',
 'Aylesbury East': 'N411',
 'Aylesbury North ': 'N412',
 'Aylesbury South': 'N413',
 'Aylesbury Town Centre': 'N106',
 'Aylesbury West': 'N414',
 'Banbury Rural': 'N425',
 'Banbury Town': 'N426',
 'Beaconsfield': 'N478',
 'Bicester Rural': 'N427',
 'Bicester Town': 'N428',
 'Bracknell Central North': 'N421',
 'Bracknell Central South': 'N422',
 'Bracknell Northern Parishes': 'N423',
 'Bracknell Town Centre': 'N289',
 'Bradwell / Stantonbury': 'N434',
 'Britwell / Haymill': 'N457',
 'Buckingham and District': 'N415',
 'Bucklebury / Downlands': 'N469',
 'Burnham / Farnhams / Taplow': 'N475',
 'Calcot / Tilehurst': 'N470',
 'Campbell Park South / Woughton': 'N435',
 'Carterton / Bampton / Burford': 'N385',
 'Central Bletchley / Fenny Stratford': 'N436',
 'Chalfonts': 'N431',
 'Chalvey / Upton / Town': 'N458',
 'Chepping Wye Valley': 'N404',


In [27]:

#Show dates available
print( api.get_dates() )

['2020-08', '2020-07', '2020-06', '2020-05', '2020-04', '2020-03', '2020-02', '2020-01', '2019-12', '2019-11', '2019-10', '2019-09', '2019-08', '2019-07', '2019-06', '2019-05', '2019-04', '2019-03', '2019-02', '2019-01', '2018-12', '2018-11', '2018-10', '2018-09', '2018-08', '2018-07', '2018-06', '2018-05', '2018-04', '2018-03', '2018-02', '2018-01', '2017-12', '2017-11', '2017-10', '2017-09']


In [28]:
neighbourhood = force.get_neighbourhood( areas['Abingdon Outer'] )
neighbourhood.priorities

[<Neighbourhood.Priority> <p>Burglary</p>,
 <Neighbourhood.Priority> <p>Violent Crime</p>,
 <Neighbourhood.Priority> <p>Anti-social Behaviour</p>]

In [29]:
neighbourhood2 = force.get_neighbourhood( areas['Abingdon Town'] )
neighbourhood2.priorities

[<Neighbourhood.Priority> <p>Burglary</p>,
 <Neighbourhood.Priority> <p>Violent Crime</p>,
 <Neighbourhood.Priority> <p>Anti-social behaviour</p>]

In [36]:
boundary = neighbourhood.boundary
boundary2 = neighbourhood2.boundary

In [37]:
def geoJsonFromNeighbourhoodBoundary(boundary):
    ''' Generate geojson polygon from police API boundary. '''
    geojson = {"type": "FeatureCollection",
               "features": [
                   {
                       "type": "Feature",
                       "properties": {},
                       "geometry": {
                           "type": "Polygon",
                           "coordinates":  [[[ll[1],ll[0]] for ll in boundary]]
                       }
                   }
               ]
              }
    return geojson
#geojson

In [38]:
import folium
import json

m = folium.Map(
    location=boundary[0], #Just pick the first point in boundary to centre map - rough and ready!
    #Note that the default location is in lat/lon order
    tiles='Mapbox Bright',
    zoom_start=11
)

folium.GeoJson(
    geoJsonFromNeighbourhoodBoundary(boundary),
    name='geojson'
).add_to(m)

folium.GeoJson(
    geoJsonFromNeighbourhoodBoundary(boundary2),
    name='geojson2'
).add_to(m)

m

In [33]:
from ipywidgets import widgets, interact


#Note that we are seeding both widgets with values from the API

force_widget = widgets.Select(options=getIdName(forces),
                              description='Police Force:',
                              disabled=False )

area_widget = widgets.Select(options=getNameId(force.neighbourhoods),
                              description='Neigbourhood areas:',
                              disabled=False )

def update_area_widget(*args):
    ''' Update neighbourhoods widget. Also update the force global variable. '''
    global force
    force = api.get_force(force_widget.value)
    area = getNameId(force.neighbourhoods)
    area_widget.options = getNameId(force.neighbourhoods)
    
force_widget.observe(update_area_widget, 'value')

def mapper(force_i, area_i):
    ''' Generate map for selected area. Also set global boundary for use elsewhere. '''
    global boundary
    neighbourhood = force.get_neighbourhood( area_widget.value )
    boundary = neighbourhood.boundary
    m = createFoliumMap(boundary)
    print(force_i, area_i)
    return m
    
interact(mapper,force_i=force_widget, area_i=area_widget);

interactive(children=(Select(description='Police Force:', options={'Avon and Somerset Constabulary': 'avon-and…

In [54]:
crimes = api.get_crimes_area(neighbourhood.boundary, date='2019-01')
crimes2 = api.get_crimes_area(neighbourhood2.boundary, date='2019-01')


In [55]:
crimes[0].location.id, crimes[0].location.latitude, crimes[0].location.longitude, crimes[0].location.name

(1203149, '51.715803', '-1.243142', 'On or near River View')

In [56]:
crimes2[0].location.id, crimes2[0].location.latitude, crimes2[0].location.longitude, crimes2[0].location.name

(1202434, '51.680053', '-1.289240', 'On or near Brampton Close')

In [57]:

#Upack category details
crimes[0].category.id, crimes[0].category.name

('anti-social-behaviour', 'Anti-social behaviour')

In [58]:
crimes2[0].category.id, crimes2[0].category.name

('anti-social-behaviour', 'Anti-social behaviour')

In [59]:
import pandas as pd


def setCrimesAsDataFrame(crimes, df=None):
    ''' Convert crimes result to dataframe. '''
    if df is None:
        df=pd.DataFrame(columns = ['cid','type', 'month','name','lid','location','lat','lon'])
        #[int, object, object, int, object, float, float]
    for c in crimes:
        df = df.append({'cid':c.id,'type':c.category.id, 'month':c.month, 'name':c.category.name,
                            'lat':c.location.latitude,'lon':c.location.longitude, 
                        'lid':c.location.id,'location':c.location.name }, ignore_index=True)

    df['lat']=df['lat'].astype(float)
    df['lon']=df['lon'].astype(float)
    
    return df

In [79]:
df = setCrimesAsDataFrame(crimes)
df.head()

,cid,type,month,name,lid,location,lat,lon
0,71357302,anti-social-behaviour,2019-01,Anti-social behaviour,1203149,On or near River View,51.715803,-1.243142
1,71357618,anti-social-behaviour,2019-01,Anti-social behaviour,1204410,On or near Kenilworth Road,51.731938,-1.331346
2,71356895,anti-social-behaviour,2019-01,Anti-social behaviour,1200398,On or near Park/open Space,51.646745,-1.294400
3,71357059,anti-social-behaviour,2019-01,Anti-social behaviour,1204543,On or near Stone Close,51.752021,-1.310551
4,71358166,anti-social-behaviour,2019-01,Anti-social behaviour,1200222,On or near Tyrrell's Way,51.630797,-1.286022


In [80]:
df2 = setCrimesAsDataFrame(crimes2)
df2.head()

,cid,type,month,name,lid,location,lat,lon
0,71356915,anti-social-behaviour,2019-01,Anti-social behaviour,1202434,On or near Brampton Close,51.680053,-1.289240
1,71356838,anti-social-behaviour,2019-01,Anti-social behaviour,1201515,On or near Supermarket,51.671086,-1.282873
2,71357346,anti-social-behaviour,2019-01,Anti-social behaviour,1201484,On or near Wharf Close,51.665776,-1.284866
3,71358117,anti-social-behaviour,2019-01,Anti-social behaviour,1202381,On or near Gibson Close,51.688939,-1.274938
4,71358139,anti-social-behaviour,2019-01,Anti-social behaviour,1201468,On or near Petrol Station,51.673435,-1.280305


In [82]:
MergeBoth = pd.concat([df, df2], axis=0)

MergeBoth.head()

,cid,type,month,name,lid,location,lat,lon
0,71357302,anti-social-behaviour,2019-01,Anti-social behaviour,1203149,On or near River View,51.715803,-1.243142
1,71357618,anti-social-behaviour,2019-01,Anti-social behaviour,1204410,On or near Kenilworth Road,51.731938,-1.331346
2,71356895,anti-social-behaviour,2019-01,Anti-social behaviour,1200398,On or near Park/open Space,51.646745,-1.294400
3,71357059,anti-social-behaviour,2019-01,Anti-social behaviour,1204543,On or near Stone Close,51.752021,-1.310551
4,71358166,anti-social-behaviour,2019-01,Anti-social behaviour,1200222,On or near Tyrrell's Way,51.630797,-1.286022


In [91]:
#Recall that the Heatmap plugin expects data in the form [[Latitude, Longitude, count], ...]

MainsDB = MergeBoth.groupby(['lat','lon']).size().reset_index().values.tolist()
MainsDB[:4]

[[51.630797, -1.286022, 3.0],
 [51.631053, -1.283692, 1.0],
 [51.63512, -1.276808, 1.0],
 [51.637321, -1.294286, 1.0]]

In [92]:
from folium.plugins import HeatMap


m = folium.Map(MainsDB[0][:2], zoom_start=12)


HeatMap(MainsDB).add_to(m)

folium.GeoJson(
    geoJsonFromNeighbourhoodBoundary(boundary),
    name='geojson',
    
).add_to(m)
folium.GeoJson(
    geoJsonFromNeighbourhoodBoundary(boundary2),
    name='geojson',
    
).add_to(m)


m

In [129]:

CSV = pd.read_csv("ThamesValleyCRIMEDATA.csv ")

FileNotFoundError: [Errno 2] File ThamesValleyCRIMEDATA.csv  does not exist: 'ThamesValleyCRIMEDATA.csv '

In [126]:
df = pd.read_html(CSV)[0] #read url table as df
#df.columns = ['PostalCode', 'Borough', 'Neighborhood'] #renaming of the columns
#df = df[df['Borough'] != 'Not assigned'] #remove boroughs 'not assigned'
#df['Neighborhood'] = np.where(df['Neighborhood'] == 'Not assigned', df['Borough'], df['Neighborhood'])
print(df)

NameError: name 'CSV' is not defined

In [95]:
crime_types = MergeBoth[['name','type']].drop_duplicates().set_index('name')['type'].to_dict()
crime_types


{'Anti-social behaviour': 'anti-social-behaviour',
 'Bicycle theft': 'bicycle-theft',
 'Burglary': 'burglary',
 'Criminal damage and arson': 'criminal-damage-arson',
 'Other theft': 'other-theft',
 'Public order': 'public-order',
 'Shoplifting': 'shoplifting',
 'Vehicle crime': 'vehicle-crime',
 'Violence and sexual offences': 'violent-crime',
 'Other crime': 'other-crime',
 'Drugs': 'drugs',
 'Theft from the person': 'theft-from-the-person'}

In [98]:
dates_Years = [d for d in api.get_dates() if '2018' or '2019' in d]
dates_Years

['2020-08',
 '2020-07',
 '2020-06',
 '2020-05',
 '2020-04',
 '2020-03',
 '2020-02',
 '2020-01',
 '2019-12',
 '2019-11',
 '2019-10',
 '2019-09',
 '2019-08',
 '2019-07',
 '2019-06',
 '2019-05',
 '2019-04',
 '2019-03',
 '2019-02',
 '2019-01',
 '2018-12',
 '2018-11',
 '2018-10',
 '2018-09',
 '2018-08',
 '2018-07',
 '2018-06',
 '2018-05',
 '2018-04',
 '2018-03',
 '2018-02',
 '2018-01',
 '2017-12',
 '2017-11',
 '2017-10',
 '2017-09']

In [101]:
crimes_All= pd.DataFrame()

for d in dates_Years:
    tmp = api.get_crimes_area(boundary, date=d)
    crimes_All = pd.concat([crimes_All, setCrimesAsDataFrame(tmp)])
    
crimes_All.head()

,cid,type,month,name,lid,location,lat,lon
0,86061326,anti-social-behaviour,2020-08,Anti-social behaviour,1208644,On or near Glebe Close,51.835287,-0.638018
1,86061004,anti-social-behaviour,2020-08,Anti-social behaviour,1210358,On or near Park/open Space,51.946490,-0.703851
2,86060136,anti-social-behaviour,2020-08,Anti-social behaviour,1209994,On or near Redwood Close,51.897264,-0.718799
3,86061605,anti-social-behaviour,2020-08,Anti-social behaviour,1208635,On or near Maud Jane's Close,51.838668,-0.631864
4,86061968,anti-social-behaviour,2020-08,Anti-social behaviour,1183686,On or near Newton Road,51.957982,-0.765924


In [102]:
crimes_All2= pd.DataFrame()

for d in dates_Years:
    tmp = api.get_crimes_area(boundary2, date=d)
    crimes_All2 = pd.concat([crimes_All2, setCrimesAsDataFrame(tmp)])
    
crimes_All2.head()

,cid,type,month,name,lid,location,lat,lon
0,86060551,anti-social-behaviour,2020-08,Anti-social behaviour,1202422,On or near Southmoor Way,51.677685,-1.291678
1,86059616,anti-social-behaviour,2020-08,Anti-social behaviour,1201206,On or near The Hyde,51.661900,-1.296811
2,86059030,anti-social-behaviour,2020-08,Anti-social behaviour,1201206,On or near The Hyde,51.661900,-1.296811
3,86059277,anti-social-behaviour,2020-08,Anti-social behaviour,1201206,On or near The Hyde,51.661900,-1.296811
4,86059090,anti-social-behaviour,2020-08,Anti-social behaviour,1201206,On or near The Hyde,51.661900,-1.296811


In [104]:
Todos_crimes=pd.concat([crimes_All,crimes_All2],axis=0)

In [112]:
overall_data = []

for d in dates_Years:
    tmp_crimes = api.get_crimes_area(boundary, date=d)
    df = setCrimesAsDataFrame(tmp_crimes)
    MainsDB = MergeBoth.groupby(['lat','lon']).size().reset_index().values.tolist()
    overall_data.append(MainsDB)

In [111]:
from folium.plugins import HeatMapWithTime

m = folium.Map(overall_data[0][0][:2], tiles='stamentoner', zoom_start=13)

HeatMapWithTime(overall_data).add_to(m)

m

In [115]:
#For a particular crime type, get the count of crimes by location
crimes = setCrimesAsDataFrame( api.get_crimes_area(boundary, date='2019-01') )
public_order_crime = crimes[crimes['type']=='public-order']
public_order_crime_count_by_location = public_order_crime.groupby(['lat','lon']).size().to_frame('Count').reset_index()
public_order_crime_count_by_location.head()

,lat,lon,Count
0,51.835283,-0.632315,2
1,51.887907,-0.690498,1
2,51.897264,-0.718799,1
3,51.897377,-0.720918,1
4,51.899269,-0.718742,1


In [116]:
#For a particular crime type, get the count of crimes by location
crimes2 = setCrimesAsDataFrame( api.get_crimes_area(boundary2, date='2019-01') )
public_order_crime2 = crimes[crimes['type']=='public-order']
public_order_crime_count_by_location2 = public_order_crime.groupby(['lat','lon']).size().to_frame('Count').reset_index()
public_order_crime_count_by_location2.head()

,lat,lon,Count
0,51.835283,-0.632315,2
1,51.887907,-0.690498,1
2,51.897264,-0.718799,1
3,51.897377,-0.720918,1
4,51.899269,-0.718742,1


In [117]:
#For convenience, let's go back to using the df name...
df = public_order_crime_count_by_location.reset_index()

In [118]:
#https://geopandas.readthedocs.io/en/latest/gallery/create_geopandas_from_pandas.html
import geopandas
from shapely.geometry import Point

df['Coordinates'] = list(zip(df['lon'], df['lat']))
df['Coordinates'] = df['Coordinates'].apply(Point)

df['RCoordinates'] = list(zip(df['lat'], df['lon']))
df['RCoordinates'] = df['RCoordinates'].apply(Point)
gdf = geopandas.GeoDataFrame(df, geometry='Coordinates')

#Avoid ambiguity about whether index is int or string
gdf['index'] = gdf['index'].apply(lambda x: 'a_{}'.format(x))

gdf.head()

,index,lat,lon,Count,Coordinates,RCoordinates
0,a_0,51.835283,-0.632315,2,POINT (-0.63231 51.83528),POINT (51.835283 -0.632315)
1,a_1,51.887907,-0.690498,1,POINT (-0.69050 51.88791),POINT (51.887907 -0.6904979999999999)
2,a_2,51.897264,-0.718799,1,POINT (-0.71880 51.89726),POINT (51.897264 -0.718799)
3,a_3,51.897377,-0.720918,1,POINT (-0.72092 51.89738),POINT (51.897377 -0.7209179999999999)
4,a_4,51.899269,-0.718742,1,POINT (-0.71874 51.89927),POINT (51.899269 -0.718742)


In [123]:
from folium.plugins import HeatMap


m = createFoliumMap(gdf)


HeatMap(df).add_to(m)

folium.GeoJson(
    geoJsonFromNeighbourhoodBoundary(boundary),
    name='geojson',
    
).add_to(m)
folium.GeoJson(
    geoJsonFromNeighbourhoodBoundary(boundary2),
    name='geojson',
    
).add_to(m)


m

NameError: name 'createFoliumMap' is not defined